# RAG demo with OpenVINO Model Server and langchain
This demo shows how to use Retrieval Augmented Generation with langchain and OpenAI API endpoint.

It assumes the model server is already deployed on the same machine on port 8000 with model meta-llama/Meta-Llama-3-8B-Instruct.

Check https://github.com/openvinotoolkit/model_server/tree/main/demos/continuous_batching to see how it can be deployed.

Target host, port and model and all the commands can be adjusted to play with it.



In [1]:
!pip install -q -r requirements.txt

In [2]:
import os

from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.prompts import PromptTemplate

# Document Splitter
from typing import List
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter, MarkdownTextSplitter
from langchain_community.document_loaders import (
    CSVLoader,
    EverNoteLoader,
    PDFMinerLoader,
    TextLoader,
    UnstructuredEPubLoader,
    UnstructuredHTMLLoader,
    UnstructuredMarkdownLoader,
    UnstructuredODTLoader,
    UnstructuredPowerPointLoader,
    UnstructuredWordDocumentLoader, )

from langchain_community.vectorstores import Chroma
from langchain.docstore.document import Document

The documents to scan with knowledge context are to be placed in ./docs folder

In [3]:
TARGET_FOLDER = "./docs/"

TEXT_SPLITERS = {
    "Character": CharacterTextSplitter,
    "RecursiveCharacter": RecursiveCharacterTextSplitter,
    "Markdown": MarkdownTextSplitter,
}

LOADERS = {
    ".csv": (CSVLoader, {}),
    ".doc": (UnstructuredWordDocumentLoader, {}),
    ".docx": (UnstructuredWordDocumentLoader, {}),
    ".enex": (EverNoteLoader, {}),
    ".epub": (UnstructuredEPubLoader, {}),
    ".html": (UnstructuredHTMLLoader, {}),
    ".md": (UnstructuredMarkdownLoader, {}),
    ".odt": (UnstructuredODTLoader, {}),
    ".pdf": (PDFMinerLoader, {}),
    ".ppt": (UnstructuredPowerPointLoader, {}),
    ".pptx": (UnstructuredPowerPointLoader, {}),
    ".txt": (TextLoader, {"encoding": "utf8"}),
}

In [4]:
!curl https://docs.openvino.ai/2024/ovms_what_is_openvino_model_server.html --create-dirs -o ./docs/ovms_what_is_openvino_model_server.html
!curl https://docs.openvino.ai/2024/ovms_docs_metrics.html -o ./docs/ovms_docs_metrics.html
!curl https://docs.openvino.ai/2024/ovms_docs_streaming_endpoints.html -o ./docs/ovms_docs_streaming_endpoints.html
!curl https://docs.openvino.ai/2024/ovms_docs_target_devices.html -o ./docs/ovms_docs_target_devices.html


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  697k    0  697k    0     0  1206k      0 --:--:-- --:--:-- --:--:-- 1204k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  719k    0  719k    0     0  1263k      0 --:--:-- --:--:-- --:--:-- 1266k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  700k    0  700k    0     0  2103k      0 --:--:-- --:--:-- --:--:-- 2098k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  717k    0  717k    0     0  1662k      0 --:--:-- --:--:-- --:--:-- 1665k


In [5]:
def load_single_document(file_path: str) -> List[Document]:
    """
    helper for loading a single document

    Params:
      file_path: document path
    Returns:
      documents loaded

    """
    ext = "." + file_path.rsplit(".", 1)[-1]
    if ext in LOADERS:
        loader_class, loader_args = LOADERS[ext]
        loader = loader_class(file_path, **loader_args)
        return loader.load()

    raise ValueError(f"File does not exist '{ext}'")

In [6]:
embeddings=HuggingFaceEmbeddings(
            model_name="sentence-transformers/all-mpnet-base-v2",
            model_kwargs={"device":"cpu"},
            show_progress=True
            )


/home/dtrawins/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
documents = []
for file_path in os.listdir(TARGET_FOLDER):
    if not file_path.endswith('.html'):
        continue
    abs_path = os.path.join(TARGET_FOLDER, file_path)
    print(f"Reading document {abs_path}...", flush=True)
    documents.extend(load_single_document(abs_path))

Reading document ./docs/ovms_docs_streaming_endpoints.html...
Reading document ./docs/ovms_docs_metrics.html...
Reading document ./docs/ovms_what_is_openvino_model_server.html...
Reading document ./docs/ovms_docs_target_devices.html...


In [8]:
spliter_name = "RecursiveCharacter"  # PARAM
chunk_size=1000  # PARAM
chunk_overlap=200  # PARAM
text_splitter = TEXT_SPLITERS[spliter_name](chunk_size=chunk_size, chunk_overlap=chunk_overlap)

texts = text_splitter.split_documents(documents)



In [9]:
try:
    db.delete_collection()
except:
    pass
db = Chroma.from_documents(texts, embeddings)

Batches: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:04<00:00,  2.07s/it]


The commands below can be used to test the retriever. It can report the content for a given query.

In [10]:
vector_search_top_k = 4
retriever = db.as_retriever(search_kwargs={"k": vector_search_top_k})

retrieved_docs = retriever.invoke("What metrics are supported in the model server?")
print(retrieved_docs[0])
print(retrieved_docs[1])
print(retrieved_docs[2])
print(retrieved_docs[3])

Batches: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 57.39it/s]

page_content='Metrics¶\n\nIntroduction¶\n\nThis document describes how to use metrics endpoint in the OpenVINO Model Server. They can be applied for:\n\nProviding performance and utilization statistics for monitoring and benchmarking purposes\n\nAuto scaling of the model server instances in Kubernetes and OpenShift based on application related metrics\n\nBuilt-in metrics allow tracking the performance without any extra logic on the client side or using network traffic monitoring tools like load balancers or reverse-proxies.\n\nIt also exposes metrics which are not related to the network traffic.\n\nFor example, statistics of the inference execution queue, model runtime parameters etc. They can also track the usage based on model version, API type or requested endpoint methods.\n\nOpenVINO Model Server metrics are compatible with Prometheus standard\n\nThey are exposed on the /metrics endpoint.\n\nAvailable metrics families¶\n\nMetrics from default list are enabled with the metrics_enab

Change the base url and model name depending on the model server deployment and configuration. It is important to use /v3/ part which is specific for the OpenVINO Model Server

In [11]:
llm = ChatOpenAI(
    openai_api_key="EMPTY",
    openai_api_base="http://localhost:8000/v3",
    model_name="meta-llama/Meta-Llama-3-8B-Instruct",
    temperature=0.0,
    seed=5,
    verbose=True
)


/home/dtrawins/.local/lib/python3.10/site-packages/langchain_core/utils/utils.py:161: UserWarning: WARNING! seed is not default parameter.
                seed was transferred to model_kwargs.
                Please confirm that seed is what you intended.
  warnings.warn(


In [12]:

prompt=PromptTemplate(input_variables=['context', 'question'], 
                      template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know.\nQuestion: {question} \nContext: {context} \nAnswer:")

print("prompt", prompt)
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

prompt input_variables=['context', 'question'] template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know.\nQuestion: {question} \nContext: {context} \nAnswer:"


Below you can start the RAG chain using your own query. It will call the embedding model first, retreive the relevant context and pass it to the LLM endpoint in a single request

In [13]:
for chunk in rag_chain.stream("Which metrics are supported in the model server? Give examples."):
    print(chunk, end="", flush=True)

Batches: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 32.00it/s]


According to the provided context, the OpenVINO Model Server supports the following metrics:

1. Gauge metrics:
	* `ovms_streams`: Number of OpenVINO execution streams
	* `ovms_current_requests`: Number of requests being currently processed by the model server
2. Counter metrics:
	* `ovms_requests_success`: Number of successful requests to a model or a DAG
	* `ovms_requests_fail`: Number of failed requests to a model or a DAG
3. Histogram metrics:
	* `ovms_request_time_us`: Processing time of requests to a model or a DAG
	* `ovms_inference_time_us`: Inference execution time in the OpenVINO backend
	* `ovms_wait_for_infer_req_time_us`: Request waiting time in the scheduling queue

These metrics are exposed on the `/metrics` endpoint and are compatible with the Prometheus 